In [1]:
#Importing the standard libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

#Importing the solver modules
import system
from obe import obe 
from states import SigmaLevel,PiLevelParity
from obe import Excitation
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
b=system.System([0,2],['1/2-'],B_field = [0.0,0.0,61.0],ignore_mF = False)

b.sigma_Hamiltonian.generate_bare()
b.sigma_Hamiltonian.Zeeman.generate_Zeeman()
b.pi_Hamiltonian.generate_bare()
b.pi_Hamiltonian.Zeeman.generate_Zeeman()

#Next diagonalize the Hamiltonian for this system
b.sigma_Hamiltonian.diagonalize()
b.pi_Hamiltonian.diagonalize()


G = b.sigma_Hamiltonian.diagonalized_states
GH = np.round(b.sigma_Hamiltonian.diagonalized_Hamiltonian,3)

## Delete the state originally numbered G[13] : mF = -3 and G[23] : mF = +3
G.pop(23)
G.pop(13)

GH = np.delete(GH,23,0); GH = np.delete(GH,23,1)
GH = np.delete(GH,13,0); GH = np.delete(GH,13,1)

E =b.pi_Hamiltonian.diagonalized_states
EH = np.round(b.pi_Hamiltonian.diagonalized_Hamiltonian,3)

#b.generate_branching_ratios(G,E)
#BR = b.branching_ratios
#np.round(BR,3)

# Edit entries in GH and EH

#GH -= np.amin(np.diag(GH))*np.identity(np.shape(GH)[0])
#EH -= np.amin(np.diag(EH))*np.identity(np.shape(EH)[0])

In [3]:
GH[0:4,0:4] -= np.amin(np.diag(GH[0:4,0:4]))*np.identity(np.shape(GH[0:4,0:4])[0])
GH[4:,4:] -= (np.amin(np.diag(GH[4:,4:]))-500)*np.identity(np.shape(GH[4:,4:])[0])

In [4]:
GH[19,19]-GH[18,18]

(20.235999999997148+0j)

In [22]:
G[4]

(-0.9683+0j) |G = 0.5, N = 2, F1 = 1.5, F = 2.0, mF = 2.0> + 
(-0.1783+0j) |G = 0.5, N = 2, F1 = 2.5, F = 2.0, mF = 2.0> + 
(0.1749+0j) |G = 0.5, N = 2, F1 = 2.5, F = 3.0, mF = 2.0>

In [6]:
H0 = np.zeros((len(G)+len(E),len(G)+len(E)),dtype=np.complex_)
H0[0:len(G),0:len(G)] = GH
H0[len(G):,len(G):] = EH
b.generate_interaction_Hamiltonian(G,E)
Hint_1=np.round(b.interaction_Hamiltonian,3)
n0 = [1/len(G)]*len(G)+[0]*len(E)

b.generate_interaction_Hamiltonian(G,E,pol=+1)
Hint_2 = np.round(b.interaction_Hamiltonian,3)

b.generate_interaction_Hamiltonian(G,E,pol=-1)
Hint_3 = np.round(b.interaction_Hamiltonian,3)

In [7]:
level_to_optimize = 3
myList_1 = []
myList_2 = []
for i in range(len(G)):
    if i != level_to_optimize:
        tempList_1 = []
        tempList_2 = []
        for j in range(len(E)):
            #look at the Hint element for z pol light
            if np.abs(Hint_1[i,len(G)+j])> 1e-2:
                tempList_1.append((i,j))
            if np.abs(Hint_2[i,len(G)+j])> 1e-2 or np.abs(Hint_3[i,len(G)+j])> 1e-2:
                tempList_2.append((i,j))
            
        myList_1.append(tempList_1)
        myList_2.append(tempList_2)

In [8]:
myList_2

[[(0, 1), (0, 3)],
 [(1, 0), (1, 2)],
 [(2, 1), (2, 3)],
 [(4, 3)],
 [(5, 0), (5, 2)],
 [(6, 0), (6, 2)],
 [(7, 1), (7, 3)],
 [(8, 1), (8, 3)],
 [(9, 0), (9, 2)],
 [(10, 1)],
 [(11, 0)],
 [(12, 1)],
 [(13, 0), (13, 2)],
 [(14, 1)],
 [(15, 1), (15, 3)],
 [(16, 0), (16, 2)],
 [(17, 1), (17, 3)],
 [(18, 0), (18, 2)],
 [(19, 0), (19, 2)],
 [(20, 3)],
 [(21, 3)]]

In [9]:
#to select a random neighbour
Gamma = 2*np.pi*2.7
tsigma = 1.0*100/Gamma/4 #2 is the number of passes
import random
def randomize_transition(myList,polarization):
    """ returns a list where each element is  tuple of ground and excited state index for the transition"""
    candidate_fields = []
    for item in myList:
        if len(item) != 0:
            candidate_fields.append(random.choice(item))
    #print(candidate_fields)
    field = []
    for item in candidate_fields:
        if len(item) != 0:
            (i,j) = item
        else:
            continue
        rabi = random.uniform(0.1, 5.0)*Gamma
        pol = polarization
        groundState = G[i]
        excitedState = E[j]
        det = random.uniform(-1,1)*Gamma
        pos = 0
        dia = 4*tsigma
        temp_field = Excitation(rabi,pol,groundState,excitedState,detuning = det,position = pos,diameter = dia ,shape = "Gaussian")
        field.append(temp_field)
    
    return field,candidate_fields


In [10]:
def flatten(x):
    temp_list = []
    for item in x:
        if type(item) == list:
            for temp_item in item:
                temp_list.append(temp_item)
        else:
            temp_list.append(item)
    return temp_list


In [11]:
import copy
current_field,transitions_Z = randomize_transition(myList_1,0)
N_z= len(current_field)
temp_field,transitions_sigma = randomize_transition(myList_2,1)
current_field.append(temp_field)
N_sigma = len(current_field[-1])
current_field=flatten(current_field)

In [12]:
transitions_Z

[(0, 0),
 (1, 1),
 (2, 2),
 (5, 3),
 (6, 3),
 (7, 2),
 (8, 0),
 (9, 1),
 (11, 1),
 (13, 1),
 (15, 0),
 (16, 1),
 (17, 2),
 (18, 3),
 (19, 3)]

In [13]:
transitions_sigma

[(0, 1),
 (1, 2),
 (2, 3),
 (4, 3),
 (5, 2),
 (6, 2),
 (7, 3),
 (8, 1),
 (9, 2),
 (10, 1),
 (11, 0),
 (12, 1),
 (13, 0),
 (14, 1),
 (15, 3),
 (16, 0),
 (17, 3),
 (18, 0),
 (19, 0),
 (20, 3),
 (21, 3)]

In [15]:
### only when we want to force a choice of transition

desired_Z = [(0, 2),(1, 1),(2, 0),(5, 3),(6, 3),(7, 2),(8, 0),(9, 1),(11, 1),(13, 1),(15, 0),(16, 1),(17, 2),(18, 3),(19, 3)] # 9 transitions
desired_sigma = [(0, 3),(1, 2),(2, 3),(4, 3),(5, 2),(6, 2),(7, 1),(8, 3),(9, 0),(10, 1),(11, 0),
                 (12, 1),(13, 0),(14, 1),(15, 1),(16, 2),(17, 1),(18, 0),(19, 2),(20, 3),(21, 3)] #11 transitions


import copy
count = 0
while True:
    current_field,transitions_Z = randomize_transition(myList_1,0)
    N_z= len(current_field)
    temp_field,transitions_sigma = randomize_transition(myList_2,1) 
    current_field.append(temp_field)
    N_sigma = len(current_field[-1])
    current_field=flatten(current_field)
    #print(count)
    count += 1
    
    if transitions_Z == desired_Z and desired_sigma == transitions_sigma:
        break
    
len(current_field) 

36

In [16]:
#Scan
def r22(rabi_det_params:list,Nz,passes=1): #rabi_det_params was a field earlier
    #modify the current fields here
    for i in range(len(current_field)):
            current_field[i].rabi = rabi_det_params[i][0]
            current_field[i].detuning = rabi_det_params[i][1]
    
    field = current_field
    package = 'Python'
    field1 = field[0:Nz]
    field2 = field[Nz:]

    X_field = []
    for field2_item in field2:
        X_field.append(field2_item)
        field2_item_pol_reversed = copy.copy(field2_item)
        field2_item_pol_reversed.pol *= -1
        X_field.append(field2_item_pol_reversed)
    
    
    steps=5000
    n=len(E)+len(G)
    r_init = np.zeros(np.shape(H0)).astype(np.complex128)
    for i in range(len(G)):
        r_init[i,i] = 1.0/len(G)
    
    Hint_Z = None
    Hint_X = None
    for count in range(passes):
        start = time.time()
        my_obe_1 = obe(field1,[G,E],H0,Hint_1,BR,Hint_func = Hint_Z)
        Hint_Z = my_obe_1.Hint
        #rotate r_init in the field basis
        
        ans = my_obe_1.solve(steps,r_init,max_step_size = 1/Gamma,package=package)
        rho = np.array(ans[-1]) #gives the solution at the end of the time
        r_init = rho.reshape(n,n)
        #stopz = time.time()
        #print(f"Z took: {stopz-start}s")
        #rotate r_init in the normal basis
    
        my_obe_2 = obe(field2,[G,E],H0,[Hint_2,Hint_3],BR,Hint_func =Hint_X)
        Hint_X = my_obe_2.Hint
        #rotate r_init in the new field_basis
        
        ans = my_obe_2.solve(steps,r_init,max_step_size = 1/Gamma,package=package)
        rho = np.array(ans[-1]) #gives the solution at the end of the time
        r_init = rho.reshape(n,n)
        #stopx = time.time()
        #print(f"X took: {stopx-stopz}s")
    
    r_level2optimize=np.real(r_init[level_to_optimize,level_to_optimize])
    rgg = 0

    for temp_count in range(len(G)):
        if temp_count != level_to_optimize:
            rgg += np.real(r_init[temp_count,temp_count])

    
    for i in range(len(G)):
        print(f"r_{i}: {np.round(r_init[i,i],5)}",end = ",")
    print("----------")
    
    return (r_level2optimize-rgg) #the new version of ax by default maximizes

# Optimization occurs here

In [ ]:
from ax.service.ax_client import AxClient

ax_client = AxClient()

#Create bounds
bounds = []
for i in range(len(current_field)):
    bounds.append({"name":"omega"+str(i),"type":"range","bounds":[0.0,15.0],"value_type":"float"})
    #bounds.append({"name":"omega"+str(i),"type":"choice","values":np.round(np.arange(0.0,25.0,0.5),1).tolist(),"value_type":"float"})
    bounds.append({"name":"det"+str(i),"type":"choice","values":np.round(np.arange(-25.0,25.0,0.5),1).tolist(),"value_type":"float"})

    
#create the constraint
rabi_per_transition = 15.0
constraint_1 = ""
constraint_2 = ""

for i in range(N_z):
    if i == N_z-1:
        constraint_1 += f"omega{i} <= {rabi_per_transition*N_z}"
    else:
        constraint_1 += f"omega{i} + "
for i in range(N_z,len(current_field)):
    if i == len(current_field)-1:
        constraint_2 += f"omega{i} <= {rabi_per_transition*(len(current_field)-N_z)}"
    else:
        constraint_2 += f"omega{i} + "

constraint_array = [constraint_1,constraint_2]    

ax_client.create_experiment(
    name = "pumping_experiment",
    parameters = bounds,
    parameter_constraints = constraint_array,
    )

In [ ]:
ax_client.get_max_parallelism()

In [ ]:
import copy
plt.ion()

arr_pop=[]

start = time.time()
import csv
with open('XZ_rabi_det_N0_3.csv','w') as file2write:
    file2write.write("Pop, (rabi, det) for the fields\n")
    file2write.write("transitions_Z :"+str(transitions_Z)+"\n")
    file2write.write("transitions_sigma :"+str(transitions_sigma)+"\n")
    for it in range(600):
        parameters,trial_index = ax_client.get_next_trial() #parameters is returned as a dict with kets omega{i} and det{i}
        rabi_det_array = [(parameters[f"omega{i}"],parameters[f"det{i}"]) for i in range(len(current_field))]
        
        pop = r22(rabi_det_array,N_z,passes=3)
        
        ax_client.complete_trial(
            trial_index = trial_index,
            raw_data = pop,
            )

        array_to_write = copy.deepcopy(rabi_det_array)
        array_to_write.insert(0,pop)
        
        #print(pop)
        file2write.write(str(array_to_write)+'\n')
        print("------------------------------------------------------------------------------")
        arr_pop.append(pop)
        plt.plot(arr_pop,'o-r')
        plt.pause(0.01)
stop = time.time()
print(f"Execution took {stop-start} s.")

In [ ]:
error

In [ ]:
start = time.time()
import csv
with open('XZ_rabi_det_N0_3_append.csv','w') as file2write:
    file2write.write("Pop, (rabi, det) for the fields\n")
    file2write.write("transitions_Z :"+str(transitions_Z)+"\n")
    file2write.write("transitions_sigma :"+str(transitions_sigma)+"\n")
    for it in range(200):
        parameters,trial_index = ax_client.get_next_trial() #parameters is returned as a dict with kets omega{i} and det{i}
        rabi_det_array = [(parameters[f"omega{i}"],parameters[f"det{i}"]) for i in range(len(current_field))]
        
        pop = r22(rabi_det_array,N_z,passes=3)
        
        ax_client.complete_trial(
            trial_index = trial_index,
            raw_data = pop,
            )

        array_to_write = copy.deepcopy(rabi_det_array)
        array_to_write.insert(0,pop)
        
        #print(pop)
        file2write.write(str(array_to_write)+'\n')
        print("------------------------------------------------------------------------------")
        arr_pop.append(pop)
        plt.plot(arr_pop,'o-r')
        plt.pause(0.01)
stop = time.time()
print(f"Execution took {stop-start} s.")

In [ ]:
import json
ax_client.save_to_json_file(filepath = "ax_client_state_N0_3.json")

In [ ]:
deliberate error here

In [17]:
f= open('XZ_rabi_det_N0_3.csv')
max_val = -1.0
max_val_array=[]
max_array = []
array_all = []
for i,line in enumerate(f):
    if i < 4:
        continue
    #print(f"read upto {i}")
    #print(line)
    mylist = eval(line)
    array_all.append(mylist)
    if mylist[0] > max_val:
        max_val = mylist[0]
        max_array = mylist
        max_val_array.append(max_val)
max_val
max_array.remove(max_val)
max_val,max_array
print(max_val)
min_array = max_array
print(max_array)
plt.plot(max_val_array)

0.09778488670521313
[(13.639505821701363, -17.0), (10.574050226567605, 24.5), (3.0900295702116995, -4.0), (10.18320687977967, -2.0), (11.348849943046162, 5.5), (3.707701566002574, -2.0), (8.194853714866145, 14.5), (5.961161883399856, -15.0), (10.32827088841913, 18.0), (12.10324066609878, -1.0), (3.8376179808265354, -6.0), (15.0, -2.5), (8.554510414178301, 20.5), (4.932135432754053, 4.0), (4.83844136801365, -5.5), (7.4660444275824105, -1.5), (6.34587805821634, 11.0), (0.6488633414511054, 3.0), (11.722423587349567, -13.0), (13.509809743141373, 19.5), (7.411518555897782, -9.0), (12.077871733584425, -10.5), (2.071609099121555, 6.5), (7.791786993681418, -9.5), (6.057971506356068, -8.5), (3.6550137762489543, -11.0), (5.646564594001668, 12.0), (12.124440035239724, 23.0), (3.1472544136554332, -0.5), (7.2850661225581135, 9.0), (2.191409055773829, 2.0), (7.544199952730505, 8.5), (12.467453792373801, 4.0), (2.9127892952284253, -8.0), (8.465818475742651, -5.0), (7.174429811646148, 11.5)]


In [ ]:
start=time.time()
r22(max_array,N_z,passes=1)
print(time.time()-start)

In [18]:
(1+max_val)/2

0.5488924433526066

## For drawing the energy levels

In [45]:
ground_mF = [item.states[0].mF+np.linspace(-1/4,1/4,3) for item in G]
ground_Energy = np.array([GH[i,i] for i in range(len(G))])
ground_Energy -= np.mean(ground_Energy)
min_ground_energy = np.amin(ground_Energy)
max_ground_energy = np.amax(ground_Energy)

excited_mF = [item.states[0].mF+np.linspace(-1/4,1/4,3) for item in E]
excited_Energy = np.array([EH[i,i] for i in range(len(E))])
excited_Energy -= np.mean(excited_Energy) - 1*np.abs(max_ground_energy-min_ground_energy)

rabi_det_params = max_array # --------the solution from the optimization-----------------
for i in range(len(current_field)):
            current_field[i].rabi = rabi_det_params[i][0]
            current_field[i].detuning = rabi_det_params[i][1]
            
#Plot the energy levels

def plot_energy_levels():
    for i in range(len(G)):
        plt.plot(ground_mF[i],ground_Energy[i]+np.zeros(3))
        plt.text(ground_mF[i][1]+0.0*np.random.random(),ground_Energy[i]-10,f"G[{i}]",fontsize = 9)
    for j in range(len(E)):
        plt.plot(excited_mF[j],excited_Energy[j]+np.zeros(3))
        plt.text(excited_mF[j][1]+0.0*np.random.random(),excited_Energy[j]-10,f"E[{j}]",fontsize = 9)
    plt.text(1.8,800,r"$J = 1/2^-$",fontsize = 12,color='red')
    plt.text(1.8,170,r"$\~{J} = 5/2$",fontsize = 12,color='red')
    plt.text(1.8,-50,r"$\~{J} = 3/2$",fontsize = 12,color='red')
    plt.text(1.8,-500,r"$J = 1/2$",fontsize = 12,color='red')
    plt.xlabel("mF",fontsize=14)
    plt.ylabel("Energy (not to scale)",fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

rabis = []
for i in range(0,len(max_array),2):
    rabis.append(min_array[i])
max_rabi = np.amax(np.abs(np.array(rabis)))
    
# Plot the z transitions
plot_energy_levels()
for idx,item in enumerate(transitions_Z):
    i,j = item
    offset = random.uniform(-0.25,0.25)
    x = [ground_mF[i][1]+offset,excited_mF[j][1]+offset]
    y = [ground_Energy[i],excited_Energy[j]+current_field[idx].detuning]
    alphaval = 1;#np.abs(current_field[idx].rabi/max_rabi)
    linewidthval = np.abs(current_field[idx].rabi/max_rabi*4)
    plt.plot(x,y,alpha =alphaval ,linewidth = linewidthval)
    plt.tight_layout()
    plt.title("Transitions coupled by z-polarized beam")
    plt.savefig('z_pol.png',dpi = 300)

plt.figure()
#Plot sigma transitions
plot_energy_levels()
for idx,item in enumerate(transitions_sigma):
    i,j = item
    offset = random.uniform(-0.05,0.05)
    x = [ground_mF[i][1]+offset,excited_mF[j][1]+offset]
    y = [ground_Energy[i],excited_Energy[j]+current_field[idx+N_z].detuning]
    alphaval = 1;#np.abs(current_field[idx+N_z].rabi/max_rabi)
    linewidthval = np.abs(current_field[idx+N_z].rabi/max_rabi*4)
    plt.plot(x,y,alpha =alphaval ,linewidth = linewidthval)
    plt.tight_layout()
    plt.title(r"Transitions coupled by x-polarized beam \\ (showing $\sigma^+$ or $\sigma^-$ component only")
    plt.savefig('x_pol.png',dpi = 300)
    #plt.plot(x,y,alpha = np.abs(current_field[idx+N_z].rabi/max_rabi),linewidth = np.abs(current_field[idx+N_z].rabi/max_rabi*10))

## Read from the data file from the bayesian optimization

In [ ]:
f= open('XZ_rabi_det_N0_3_office_ax_resume.csv')
all_array = []
for i,line in enumerate(f):
    if i < 4:
        continue
    mylist = eval(line)
    all_array.append(mylist)
sorted(all_array, key=lambda item: item[0])
#all_array = sorted(all_array)
#min_array = all_array[1]
#print(min_array[0])
#min_array.pop(0)
#min_array[1] = (12.65,0)
#for i in range(len(min_array)):
#    (a,b) = min_array[i]
#    min_array[i] = (round(a,1),round(b,1))
#min_array

In [ ]:
all_array[-3].remove(all_array[-3][0])
all_array[-3]

In [ ]:
for j in range(1,10):
    all_array[-j].remove(all_array[-j][0])
    print(all_array[-j])
    r22(all_array[-j],N_z,passes=3)


In [ ]:
current_field[1]

## Implement gradient descent with the optimal solution and small learning step size

In [ ]:
plt.ion()
import copy
from joblib import Parallel, delayed
import multiprocessing
N_processor = -1 #now use all the processors
#calculate gradient of p
def grad(rabi_det_params,Nz,passes=1):
    
    grad_fields = []
    temp_field = copy.deepcopy(rabi_det_params)
    
    p = r22(rabi_det_params,Nz,passes)
    step_size=0.01
    count = 0
    N=len(rabi_det_params)
    
    for i in range(N): #even incides are rabis and odd indices are detunings
        temp_field[i][0] += step_size  # The rabi frequency
        grad_fields.append([copy.copy(item) for item in temp_field])
        temp_field = copy.deepcopy(rabi_det_params)
        
        temp_field[i][1] += step_size # the detuning
        grad_fields.append([copy.copy(item) for item in temp_field])
        temp_field = copy.deepcopy(rabi_det_params)
       
    #print(grad_fields)
    increament_vals= Parallel(n_jobs = N_processor)(delayed(r22)(iteration_field,N_z,passes) for iteration_field in grad_fields)
    increament_vals = np.array(increament_vals).reshape(N,2)
    
    current_vals = p*np.ones(2*N).reshape(N,2)
    dX = step_size*np.ones(2*N).reshape(N,2)
    gradient_p = (increament_vals - current_vals)/dX
        
    return p,gradient_p
        
learn_rate = 0.003#*np.ones(len(min_array))
current_array = np.array(min_array)
"""
current_array = np.array([ 5.75788161e+00, -1.00561741e+00, -4.65819174e-02, -1.13462899e+01,
       -5.34771998e-02, -3.30470704e+00,  3.92047082e+00, -1.26412562e+00,
        9.32853136e+00, -1.80212259e+00,  6.01684382e+00, -2.08144123e+01,
        9.34802445e+00,  1.91518091e+01, -4.14091218e-02,  3.31665994e+00,
        6.65766154e+00, -1.90843448e+01,  5.84269621e+00, -1.99850969e+01,
        9.28915766e+00, -1.97303426e+01,  8.52778113e+00, -2.03186644e+01,
       -1.19106262e-02, -2.00030886e+01,  3.26836954e+00, -1.34928476e+01,
        6.67525090e-03,  8.00142427e+00,  5.54601577e+00,  1.89538351e+01,
        3.98001456e+00, -1.98822238e+01,  1.08846042e+01,  6.28568895e+00])
"""
arr=[]
iter_max = 1000
for it in range(iter_max):
    print(current_array)
    pop,gradient = grad(current_array,N_z,passes = 1)
    
    current_array -= learn_rate*gradient
    
    arr.append(pop)
    print(it,np.round(pop,3),end=",")
    plt.clf()
    plt.plot(arr,'-o')
    plt.show()
    plt.pause(0.01)

In [ ]:
transitions_Z

In [ ]:
with open('test.csv','w') as file2write:
    file2write.write("transitions_Z :"+str(transitions_Z)+"\n")
    file2write.write("transitions_sigma :"+str(transitions_sigma)+"\n")

In [ ]:
dir(transitions_Z)

In [ ]:
str(transitions_Z)